In [16]:
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
import os
import json
import math
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import soundfile
import random


import commons
import utils
from vits.data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from vits.models import SynthesizerTrn
from vits.text.symbols import symbols
from vits.text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [3]:
hps = utils.get_hparams_from_file("vits/configs/vctk_base.json")

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("/home/xd/Downloads/pretrained_vctk.pth", net_g, None)

/home/xd/miniconda3/envs/torch_env/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


INFO:root:Loaded checkpoint '/home/xd/Downloads/pretrained_vctk.pth' (iteration 0)


In [19]:
stn_tst = get_text("Hey jarvis", hps)
for noise_scale in np.arange(0.1, 1.0, 0.1):
  for noise_scale_w in np.arange  (0.1, 1.0, 0.1):
    with torch.no_grad():
      x_tst = stn_tst.cuda().unsqueeze(0)
      x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
      sid = torch.LongTensor([4]).cuda()
      audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=noise_scale, noise_scale_w=noise_scale_w, length_scale=2)[0][0,0].data.cpu().float().numpy()
# ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False)) 
    sf.write(f"/home/xd/Documents/wake-word/.data/wake/vits_{noise_scale:.2f}_{noise_scale_w:.2f}.wav", audio, hps.data.sampling_rate)
    print(f"Saved file at /home/xd/Documents/wake-word/.data/wake/vits_{noise_scale:.2f}_{noise_scale_w:.2f}.wav")

Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.10_0.10.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.10_0.20.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.10_0.30.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.10_0.40.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.10_0.50.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.10_0.60.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.10_0.70.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.10_0.80.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.10_0.90.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.20_0.10.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.20_0.20.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.20_0.30.wav
Saved file at /home/xd/Documents/wake-word/.data/wake/vits_0.20_0.40.wav
Saved file at /home/xd/Documents/wake-word/.data/wa